### Classifier and Grammatical Error Correction 102034038 湯忠憲

In [28]:
#feature extraction function
def gen_feature_set(tags, offset,words,tag_list, phrases):
    def tags_to_word_pos(tags):
        return " ".join((tag[0] + "_" + tag[1] for tag in tags))

    features = {}
    ta_offset = offset -1
    features['TGLR'] = tags_to_word_pos([tags[ta_offset - 1], tags[ta_offset + 1]]) if 0 < ta_offset < len(tags) - 1 else ''
    features['TGL'] = tags_to_word_pos([tags[ta_offset - 2], tags[ta_offset - 1]]) if 1 < ta_offset else ''
    features['TGR'] = tags_to_word_pos([tags[ta_offset + 1], tags[ta_offset + 2]]) if ta_offset < len(tags) - 2 else ''
    features['BGL'] = tags_to_word_pos([tags[ta_offset - 1]]) if 0 < ta_offset else ''
    features['BGR'] = tags_to_word_pos([tags[ta_offset + 1]]) if ta_offset < len(tags) - 1 else ''
    #robust feature
    features["Wrong_word"] = words[ta_offset]
    features["Wrong_tag"] = tag_list[ta_offset]
    
    headword_pos = ""
    fh_word = ""
    fh_tag = ""
    fp = ""
    for k, ph in enumerate(phrases[offset:]):
        if ph.startswith("I"):
            headword_pos = words[k+offset]+"_"+tag_list[k+offset]
            fh_word = words[k+offset]
            fh_tag = tag_list[k+offset]
            fp = " ".join((words[b]+"_"+tag_list[b] for b in range(offset,offset+k+1)))
            break
    features["FH"] = headword_pos
    features["FHword"] = fh_word
    features["FHtag"] = fh_tag
    features["FP"] = fp
    phr_pre = ""
    for p, ph in enumerate(reversed(phrases[:offset-1])):
        if ph.startswith("B"):
            phr_pre = ph[-2:]
            break
    pv=""
    pv_tag=""
    for h, tag in enumerate(reversed(tag_list[:offset-1])):
        if tag.startswith("V"):
            pv= words[offset-h-2]+"_"+tag_list[offset-h-2]
            pv_tag = tag_list[offset -h -2]
            break
    features["pv"] = pv
    features["pv_tag"] = pv_tag
    features["PHR_pre"] = phr_pre
    return features

Above is a feature generator, I implement 12 features from Table 1. Besides, I recorded the original **wrong word** and **PoS tags** as features and found out that they're robust !

In [63]:
from __future__ import division
from sklearn.linear_model import LogisticRegression
from nltk.classify.scikitlearn import SklearnClassifier



corrections = list(open('wiki.prep.corrections.clean.100k'))
data = list(open("wiki.prep.sents.clean.genia.100k"))

size = len(data)
#split data
train_data = data[:-(size//10)]
test_data = data[-(size//10):]
train_correction = corrections[:-(size//10)]
test_correction = corrections[-(size//10):]

In [36]:
#feature extraction test

index = 1
lines = train_data[index].split("\t")
words = lines[1].split()
print words,len(words)
tags = lines[2].split()
print tags,len(tags)
phrases = lines[3].split()
print phrases,len(phrases)
input_tags = [(words[i],tags[i],phrases[i]) for i in range(len(tags))]
offset = int(train_correction[index].split("\t")[0])
label = train_correction[index].split("\t")[3]

print "wrong word:",words[offset-1]
print "answer:", label
print "phrase:", phrases[offset-1]
for i, ph in enumerate(phrases[offset:]):
    if ph.startswith("I"):
        print words[i+offset]+"_"+tags[i+offset]
        print "following range",range(offset, offset+i+1)
        print " ".join((words[b]+"_"+tags[b] for b in range(offset,offset+i+1)))
        break
for i, ph in enumerate(reversed(phrases[:offset-1])):
    if ph.startswith("B"):
        print "i=",i
        print "ph=", ph
        print words[offset-i-2]+"_"+tags[offset-i-2],ph[-2:]
        break
for h, tag in enumerate(reversed(tags[:offset-1])):
    if tag.startswith("V"):
        print "h=",h
        print words[offset-h-2]+"_"+tags[offset-h-2]
        break
#print input_tags
#gen_feature_set(input_tags,offset)

['In', '1927', ',', 'while', 'work', 'for', 'Bell', 'Labs', ',', 'Davisson', 'and', 'Lester', 'Germer', 'perform', 'an', 'experiment', 'show', 'that', 'electron', 'be', 'diffract', 'from', 'the', 'surface', 'of', 'a', 'crystal', 'of', 'nickel', '.'] 30
['IN', 'CD', ',', 'IN', 'VBG', 'IN', 'NNP', 'NNP', ',', 'NNP', 'CC', 'NNP', 'NNP', 'VBD', 'DT', 'NN', 'VBG', 'IN', 'NNS', 'VBD', 'VBN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN', '.'] 30
['B-PP', 'B-NP', 'O', 'B-SBAR', 'B-VP', 'B-PP', 'B-NP', 'I-NP', 'O', 'B-NP', 'O', 'B-NP', 'I-NP', 'B-VP', 'B-NP', 'I-NP', 'B-VP', 'B-SBAR', 'B-NP', 'B-VP', 'I-VP', 'B-PP', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'O'] 30
wrong word: from
answer: at
phrase: B-PP
surface_NN
following range [22, 23]
the_DT surface_NN
i= 1
ph= B-VP
be_VBD VP
h= 0
diffract_VBN


In [64]:
#transform the raw data to feature-answer pairs

def data_conversion(data_input,correction):
    featuresets = []
    for i in range(len(data_input)):
        lines = data_input[i].split("\t")
        words = lines[1].split()
        tags = lines[2].split()
        phrases = lines[3].split()
        input_tags = [(words[j],tags[j],phrases[j]) for j in range(len(tags))]
        offset = int(correction[i].split("\t")[0])
        label = correction[i].split("\t")[3].lower() #lower the answer to reduce number of class
        featuresets.append((gen_feature_set(input_tags,offset,words,tags,phrases),label))
    return featuresets

In [65]:
trainData = data_conversion(train_data,train_correction)

In [66]:
trainData[:3]

[({'BGL': 'last_JJ',
   'BGR': 'a_DT',
   'FH': 'boot_NN',
   'FHtag': 'NN',
   'FHword': 'boot',
   'FP': 'a_DT boot_NN',
   'PHR_pre': 'JP',
   'TGL': 'style_NN last_JJ',
   'TGLR': 'last_JJ a_DT',
   'TGR': 'a_DT boot_NN',
   'Wrong_tag': 'TO',
   'Wrong_word': 'to',
   'pv': 'use_VB',
   'pv_tag': 'VB'},
  'from'),
 ({'BGL': 'diffract_VBN',
   'BGR': 'the_DT',
   'FH': 'surface_NN',
   'FHtag': 'NN',
   'FHword': 'surface',
   'FP': 'the_DT surface_NN',
   'PHR_pre': 'VP',
   'TGL': 'be_VBD diffract_VBN',
   'TGLR': 'diffract_VBN the_DT',
   'TGR': 'the_DT surface_NN',
   'Wrong_tag': 'IN',
   'Wrong_word': 'from',
   'pv': 'diffract_VBN',
   'pv_tag': 'VBN'},
  'at'),
 ({'BGL': 'part_NN',
   'BGR': 'the_DT',
   'FH': 'forthcoming_JJ',
   'FHtag': 'JJ',
   'FHword': 'forthcoming',
   'FP': 'the_DT forthcoming_JJ',
   'PHR_pre': 'NP',
   'TGL': 'take_VB part_NN',
   'TGLR': 'part_NN the_DT',
   'TGR': 'the_DT forthcoming_JJ',
   'Wrong_tag': 'IN',
   'Wrong_word': 'at',
   'pv': 'ta

In [67]:
testData= data_conversion(test_data,test_correction)

In [68]:
import nltk
# train your classifier
classifier = SklearnClassifier(LogisticRegression(C = 10))
classifier.train(trainData)

# test your classifier
print "accuracy: ",nltk.classify.accuracy(classifier, testData)

correct = 0
prediction = []
answer = []
for feature in testData:
    prediction.append(classifier.classify(feature[0]))
    answer.append(feature[1])

accuracy:  0.5418


First, I tried logistic regression as a classifier and set C, which is inverse of regularization strength, to 10. It gave acuracy about 0.47 using _wiki.prep.sents.clean.genia.10k_ data and 0.54 in using _wiki.prep.sents.clean.genia.100k_ data.
Then, I used *classification_report* function from scikit-learn. It could show f1-score for each category and the average f1-score

In [69]:
from sklearn.metrics import classification_report
import numpy as np
# np.set_printoptions(threshold=np.nan)
print classification_report(answer, prediction)

             precision    recall  f1-score   support

     """in"       1.00      1.00      1.00        11
     aboard       0.50      0.20      0.29         5
      about       0.25      0.07      0.10        46
      above       0.00      0.00      0.00         8
     across       0.79      0.42      0.55        26
      after       0.57      0.48      0.52       141
    against       0.82      0.31      0.45        58
     albeit       1.00      1.00      1.00         1
      along       0.40      0.14      0.21        14
  alongside       1.00      0.10      0.18        10
   although       0.63      0.58      0.60        78
       amid       0.00      0.00      0.00         2
     amidst       0.00      0.00      0.00         1
      among       0.62      0.56      0.59        41
    amongst       0.80      0.33      0.47        12
     around       0.64      0.26      0.37        34
         as       0.30      0.17      0.22       172
         at       0.41      0.31      0.35   

From the above table, there're lots of **null** answer in the dataset and the word **to** has lower precision and recall.

I also try other classification algorithms like **randomforest** and **svm**. After tuning the hyper-parameter, I thought the **logistic regression** should be a better choice. The F1-score is about 0.51.

In [51]:
from sklearn.ensemble import RandomForestClassifier

rf = SklearnClassifier(RandomForestClassifier(n_estimators=50,max_features=None,class_weight="balanced"))
rf.train(trainData)

# test your classifier
print "accuracy: ",nltk.classify.accuracy(rf, testData)
correct = 0
prediction = []
answer = []
for feature in testData:
    prediction.append(rf.classify(feature[0]))
    answer.append(feature[1])
    if feature[0]==feature[1]:
        correct += 1
print classification_report(answer, prediction)

accuracy:  0.395
             precision    recall  f1-score   support

        'on       0.00      0.00      0.00         1
      After       0.67      0.67      0.67         3
   Although       0.67      0.50      0.57         4
      Among       0.00      0.00      0.00         2
         As       0.00      0.00      0.00         4
         At       0.20      0.50      0.29         2
    Because       0.00      0.00      0.00         2
         By       0.00      0.00      0.00         5
    Despite       0.00      0.00      0.00         2
     During       0.20      1.00      0.33         2
        For       0.00      0.00      0.00         2
       From       0.00      0.00      0.00         3
         In       0.24      0.64      0.35        11
       NULL       0.62      0.66      0.64       333
         On       0.62      0.62      0.62         8
      Since       0.33      0.20      0.25         5
     Though       0.00      0.00      0.00         0
 Throughout       0.00      

In [27]:
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.01,0.001, 0.0001], 'kernel': ['rbf']},
 ]
svm = SklearnClassifier(GridSearchCV(SVC(),param_grid))
svm.train(trainData)

# test your classifier
print "accuracy: ",nltk.classify.accuracy(svm, testData)
correct = 0
prediction = []
answer = []
for feature in testData:
    prediction.append(rf.classify(feature[0]))
    answer.append(feature[1])
    if feature[0]==feature[1]:
        correct += 1
print classification_report(answer, prediction)

c:\python27\lib\site-packages\sklearn\cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


accuracy:  0.4
             precision    recall  f1-score   support

        'on       0.00      0.00      0.00         1
      After       0.00      0.00      0.00         3
   Although       0.00      0.00      0.00         4
      Among       0.00      0.00      0.00         2
         As       0.00      0.00      0.00         4
         At       0.00      0.00      0.00         2
    Because       0.00      0.00      0.00         2
         By       0.00      0.00      0.00         5
    Despite       0.00      0.00      0.00         2
     During       0.00      0.00      0.00         2
        For       0.00      0.00      0.00         2
       From       0.00      0.00      0.00         3
         In       0.10      0.27      0.14        11
       NULL       0.49      0.69      0.57       333
         On       0.33      0.50      0.40         8
      Since       0.00      0.00      0.00         5
 Throughout       0.00      0.00      0.00         1
     Unlike       0.00      0.